In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

##### *Requiered Functions:*

In [2]:
def create_server_connection(host_name, user_name, user_password):
    """This functions establishes the connection between Python and MySQL local server."""
    
    connection = None # To avoid having multiple connections at the same time.
    
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            passwd = user_password
        )
        print("Connection Successful.")
    
    except Error as error:
        print(f"Error: {error}!")
        
    return connection

In [3]:
def execute_query(connection, query):
    """This function executes any query without fetching any data."""
    
    cursor = connection.cursor()
    
    try:
        cursor.execute(query)
        connection.commit()
        print("Query Successful.")
    
    except Error as error:
        print(f"Error: {error}!")

In [4]:
def fetching_data(connection, query):
    """This function executes any query and fetches the data."""
    
    cursor = connection.cursor(buffered=True, dictionary=True) # To buffer every data in a dictionary format.
    
    try:
        cursor.execute(query)
        connection.commit()
        print("Data Fetched Correctly.")
    
    except Error as error:
        print(f"Error: {error}!")
        
    return cursor.fetchall() # To send every data as dictionary.

In [5]:
def execute_sql_file(connection, file_name):
    """This function executes SQL files."""
    
    assert file_name.split('.')[-1] in ["sql", "txt"], """Files MUST BE SQL or TXT!"""
    
    flag = 0
    
    cursor = connection.cursor()
    
    try:
        with open(file_name, 'r') as sql_file:
            content = sql_file.read() # To read all the content from the SQL File.
            Queries = content.split(';')[:-1] # To obtain all the queries from the file, except the garbage after the last ';'.
            for query in Queries:
                cursor.execute(query)
                connection.commit()
            flag = 1
    
    except Error as error:
        print(f"Error: {error}!")
        
    if flag:
        print("SQL File correctly executed.")

In [6]:
def sql_to_dataframes(data):
    """This function turns the data from any sql database into a pandas dataframe."""

    data_keys = [key for key in data[0].keys()]
    
    lista_valores = [[dic[key] for dic in data] for key in data_keys]
    
    data_for_dataframe = {key:lista_valores[i] for i,key in enumerate(data_keys)}
    
    sql_dataframe = pd.DataFrame(data = data_for_dataframe)
    
    return sql_dataframe, data_keys

***

***DO NOT EXECUTE THIS PART MORE THAN ONCE. IN CASE WE WANT TO RESTART THE KERNEL, WE NEED TO ERASE THE TABLE OR DATABASE.***

*Connection with MySQL local server:*

In [7]:
connection = create_server_connection("localhost", "root", "password")

Connection Successful.


*Create Database:*

In [8]:
execute_query(connection, "CREATE DATABASE Section_14y15;")

Query Successful.


*Selecting Database:*

In [9]:
execute_query(connection, "USE Section_14y15;")

Query Successful.


*Creating Tables:*

In [10]:
# TABLE No. 1:

Users_Table = """

CREATE TABLE Users (
    ID INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    User_Name VARCHAR (255) UNIQUE NOT NULL,
    Created_At TIMESTAMP DEFAULT NOW() ON UPDATE NOW() 
);

"""

execute_query(connection, Users_Table)

Query Successful.


In [11]:
# TABLE No. 2:

Photos_Table = """

CREATE TABLE Photos (
    ID INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    Image_URL VARCHAR (255) NOT NULL,
    User_ID INT NOT NULL,
    Created_At TIMESTAMP DEFAULT NOW() ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY (User_ID) REFERENCES Users(ID) ON DELETE CASCADE
);

"""

execute_query(connection, Photos_Table)

Query Successful.


In [12]:
# TABLE No. 3:

Comments_Table = """

CREATE TABLE Comments (
    ID INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    Comment_Text VARCHAR (255) NOT NULL,
    Photo_ID INT NOT NULL,
    User_ID INT NOT NULL,
    Created_At TIMESTAMP DEFAULT NOW() ON UPDATE CURRENT_TIMESTAMP(),
    FOREIGN KEY (User_ID) REFERENCES Users(ID) ON DELETE CASCADE,
    FOREIGN KEY (Photo_ID) REFERENCES Photos(ID) ON DELETE CASCADE
);

"""

execute_query(connection, Comments_Table)

Query Successful.


In [13]:
# TABLE No. 4:

Likes_Table = """

CREATE TABLE Likes (
    User_ID INT NOT NULL,
    Photo_ID INT NOT NULL,
    Created_At TIMESTAMP DEFAULT NOW() ON UPDATE NOW(),
    FOREIGN KEY(User_ID) REFERENCES Users(ID) ON DELETE CASCADE,
    FOREIGN KEY(Photo_ID) REFERENCES Photos(ID) ON DELETE CASCADE,
    PRIMARY KEY(User_ID, Photo_ID)
);

"""

execute_query(connection, Likes_Table)

Query Successful.


In [14]:
# TABLE No. 5:

Follow_Table = """

CREATE TABLE Follows (
    Follower_ID INT NOT NULL,
    Followee_ID INT NOT NULL,
    Create_At TIMESTAMP DEFAULT NOW() ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY(Follower_ID) REFERENCES Users(ID) ON DELETE CASCADE,
    FOREIGN KEY(Followee_ID) REFERENCES Users(ID) ON DELETE CASCADE,
    PRIMARY KEY (Follower_ID, Followee_ID)
);

"""

execute_query(connection, Follow_Table)

Query Successful.


In [15]:
# TABLE No. 6:

Tags_Table = """

CREATE TABLE Tags (
    ID INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    Tag_Name VARCHAR (255) UNIQUE NOT NULL,
    Created_At TIMESTAMP DEFAULT NOW() ON UPDATE CURRENT_TIMESTAMP
);

"""

execute_query(connection, Tags_Table)

Query Successful.


In [16]:
# TABLE No. 7:

PhotoTags_Table = """

CREATE TABLE Photo_Tags (
    Photo_ID INT NOT NULL,
    Tag_ID INT NOT NULL,
    FOREIGN KEY (Photo_ID) REFERENCES Photos(ID) ON DELETE CASCADE,
    FOREIGN KEY (Tag_ID) REFERENCES Tags(ID) ON DELETE CASCADE,
    PRIMARY KEY(Photo_ID, Tag_ID) /* To avoid the same user to tag the the same photo 
    with the same hashtag multiple times. */
);

"""

execute_query(connection, PhotoTags_Table)

Query Successful.


*Inserting Data:*

In [17]:
file_name = "ig_clone_data.sql"

execute_sql_file(connection, file_name)

SQL File correctly executed.


***

***It's time to work with the data:***

#### **Note: Do not consider the default indexes in dataframes.**  
#### **Note 2: I won't use pandas to manipulate any dataframe, I only will use SQL statements even though it would be easiest to use pandas. The purpose is to keep practicing SQL.**

*Retrieve 5 oldest users:*

In [18]:
query = """

SELECT * FROM Users ORDER BY Created_At LIMIT 5;

"""

retrieved_data = fetching_data(connection, query)

Data Fetched Correctly.


In [19]:
sql_dataframe, data_keys = sql_to_dataframes(retrieved_data)
sql_dataframe

,ID,User_Name,Created_At
0,80,Darby_Herzog,2016-05-06 00:14:21
1,67,Emilio_Bernier52,2016-05-06 13:04:30
2,63,Elenor88,2016-05-08 01:30:41
3,95,Nicole71,2016-05-09 17:30:22
4,38,Jordyn.Jacobson2,2016-05-14 07:56:26


*Find the most popular day of the week where people got registered:*

In [20]:
query = """

SELECT DATE_FORMAT(Created_At, "%W") AS "Day of the week.", COUNT(*) AS "Quantity of Days." FROM Users GROUP BY
DATE_FORMAT(Created_At, "%w") ORDER BY 2 DESC;

"""

retrieved_data = fetching_data(connection, query)

Data Fetched Correctly.


In [21]:
sql_dataframe, data_keys = sql_to_dataframes(retrieved_data)
sql_dataframe

,Day of the week.,Quantity of Days.
0,Thursday,16
1,Sunday,16
2,Friday,15
3,Tuesday,14
4,Monday,14
5,Wednesday,13
6,Saturday,12


In [22]:
query = """

SELECT DATE_FORMAT(Created_At, '%W') AS 'Day of the week.', COUNT(*) AS 'Quantity of Days.' FROM Users GROUP BY
DATE_FORMAT(Created_At,'%w') ORDER BY 2 DESC LIMIT 2;

"""

retrieved_data = fetching_data(connection, query)

Data Fetched Correctly.


In [23]:
sql_dataframe, data_keys = sql_to_dataframes(retrieved_data)
sql_dataframe

,Day of the week.,Quantity of Days.
0,Thursday,16
1,Sunday,16


So, as we can see the most popular days where people registered on was: "Thursday" and "Sunday". I didn't use pandas to just print out these days, because I want to keep practicing SQL.

*Find the Users who have never posted a photo:*

In [24]:
query = """

SELECT User_Name AS 'Users who have never posted a photo' FROM Users LEFT JOIN Photos ON Users.ID = Photos.User_ID
WHERE Image_URL IS NULL;

"""

retrieved_data = fetching_data(connection, query)

Data Fetched Correctly.


In [25]:
sql_dataframe, data_keys = sql_to_dataframes(retrieved_data)
sql_dataframe

,Users who have never posted a photo
0,Aniya_Hackett
1,Bartholome.Bernhard
2,Bethany20
3,Darby_Herzog
4,David.Osinski47
5,Duane60
6,Esmeralda.Mraz57
7,Esther.Zulauf61
8,Franco_Keebler64
9,Hulda.Macejkovic


*What and Whose photo has received the most likes?:*

In [26]:
query = """

SELECT User_Name, Photo_ID, Image_URL, COUNT(*) AS 'Number of Likes' FROM Likes INNER JOIN Photos 
ON Likes.Photo_ID = Photos.ID INNER JOIN Users ON Users.ID = Photos.User_ID GROUP BY 2 
ORDER BY 4 DESC LIMIT 1;

"""

retrieved_data = fetching_data(connection, query)

Data Fetched Correctly.


In [27]:
sql_dataframe, data_keys = sql_to_dataframes(retrieved_data)
sql_dataframe

,User_Name,Photo_ID,Image_URL,Number of Likes
0,Zack_Kemmer93,145,https://jarret.name,48


*How many times does a user post?:*

In [28]:
query = """

SELECT User_Name, COUNT(Image_URL) AS 'No. Of Posts' FROM Users LEFT JOIN Photos ON Users.ID = Photos.User_ID
GROUP BY User_Name ORDER BY 2 DESC;

"""

retrieved_data = fetching_data(connection, query)

Data Fetched Correctly.


In [29]:
sql_dataframe, data_keys = sql_to_dataframes(retrieved_data)
sql_dataframe.style.set_properties()

,User_Name,No. Of Posts
0,Eveline95,12
1,Clint27,11
2,Cesar93,10
3,Delfina_VonRueden68,9
4,Aurelie71,8
5,Jaime53,8
6,Donald.Fritsch,6
7,Adelle96,5
8,Justina.Gaylord27,5
9,Janet.Armstrong,5


*What is the total average of posted photos?:*

In [30]:
query = """

SELECT (SELECT COUNT(User_ID) FROM Photos) / (SELECT COUNT(ID) FROM Users) AS 'Average Posted Photos.'

"""

retrieved_data = fetching_data(connection, query)

Data Fetched Correctly.


In [31]:
sql_dataframe, data_keys = sql_to_dataframes(retrieved_data)
sql_dataframe

,Average Posted Photos.
0,2.5700


*What are the top 5 commonly used hashtags?:*

In [48]:
query = """

SELECT Tag_Name, COUNT(*) AS "Repetitions" FROM Photo_Tags INNER JOIN Tags ON Tags.ID = Photo_Tags.Tag_ID
GROUP BY Tag_ID ORDER BY 2 DESC LIMIT 7;

"""

retrieved_data = fetching_data(connection, query)

Data Fetched Correctly.


In [49]:
sql_dataframe, data_keys = sql_to_dataframes(retrieved_data)
sql_dataframe

,Tag_Name,Repetitions
0,smile,59
1,beach,42
2,party,39
3,fun,38
4,food,24
5,lol,24
6,concert,24


Since the number 5 would be "food, lol and concert", because those have the same number of repetitions.

*Find the bots that liked all the photos:*

In [70]:
query = """
SELECT User_Name, User_ID, COUNT(*) AS Likes_Given FROM Users INNER JOIN Likes ON 
Users.ID = Likes.User_ID GROUP BY 2 HAVING(COUNT(*) = (SELECT COUNT(DISTINCT Photo_ID) FROM Likes)) 
ORDER BY 3 DESC;

"""

retrieved_data = fetching_data(connection, query)

Data Fetched Correctly.


In [71]:
sql_dataframe, data_keys = sql_to_dataframes(retrieved_data)
sql_dataframe

,User_Name,User_ID,Likes_Given
0,Aniya_Hackett,5,257
1,Bethany20,91,257
2,Duane60,54,257
3,Jaclyn81,14,257
4,Janelle.Nikolaus81,76,257
5,Julien_Schmidt,57,257
6,Leslie67,75,257
7,Maxwell.Halvorson,24,257
8,Mckenna17,41,257
9,Mike.Auer39,66,257


In [72]:
print(f"Total Number of bots: {len(sql_dataframe)}")

Total Number of bots: 13
